In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from config import DATA, DATA_2, LAYERS_SIZE, LAYERS_SIZE_2, DEFAULT_DEPTH
from torch.utils.data import Dataset, DataLoader
from player import RandomPlayer, MiniMaxPlayer, MiniMaxPlayerWithHFunction
from torch.nn.functional import l1_loss
from mlp import FlexibleMLP
from display import GameVisualize

/Users/ntd432/Desktop/SE4GD/UnivAQ/AI/code/ai-univaq/minimax/chess_game.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load("model.pth

In [4]:
class CSVDataset(Dataset):
    def __init__(self, file_path):
        # Load the data
        data = pd.read_csv(file_path)
        
        # Separate features and target
        self.X = data.iloc[:, :-1].values  # All columns except the last one
        self.y = data.iloc[:, -1].values  # The last column
        
        # Convert to tensors
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## Task 3

### Generate train data with H0 and HL

In [ ]:
game = GameVisualize()
player_1 = RandomPlayer(True)
player_2 = MiniMaxPlayer(False, depth=DEFAULT_DEPTH, generate_data=True)
x = []
for i in range(1):
    result = game.start_game(player_1, player_2, visual=False, pause=1e-15)

In [14]:
dataset = CSVDataset(DATA)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [15]:
model = FlexibleMLP(LAYERS_SIZE)
print(model)

# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

FlexibleMLP(
  (model): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): Softmax(dim=1)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Softmax(dim=1)
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): Softmax(dim=1)
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [16]:
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "model.pth")

/Users/ntd432/Desktop/SE4GD/UnivAQ/AI/code/ai-univaq/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Users/ntd432/Desktop/SE4GD/UnivAQ/AI/code/ai-univaq/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1/100, Loss: 0.1592
Epoch 2/100, Loss: 4.1142
Epoch 3/100, Loss: 0.0645
Epoch 4/100, Loss: 0.1328
Epoch 5/100, Loss: 4.0128
Epoch 6/100, Loss: 94.0104
Epoch 7/100, Loss: 0.0174
Epoch 8/100, Loss: 0.0273
Epoch 9/100, Loss: 1.0220
Epoch 10/100, Loss: 4.9489
Epoch 11/100, Loss: 0.0340
Epoch 12/100, Loss: 0.9923
Epoch 13/100, Loss: 4.9868
Epoch 14/100, Loss: 0.0267
Epoch 15/100, Loss: 3.9769
Epoch 16/100, Loss: 5.0212
Epoch 17/100, Loss: 4.9872
Epoch 18/100, Loss: 1.0120
Epoch 19/100, Loss: 94.0042
Epoch 20/100, Loss: 0.0081
Epoch 21/100, Loss: 1.0670
Epoch 22/100, Loss: 1.0088
Epoch 23/100, Loss: 0.0807
Epoch 24/100, Loss: 3.9605
Epoch 25/100, Loss: 5.0137
Epoch 26/100, Loss: 3.9680
Epoch 27/100, Loss: 3.9990
Epoch 28/100, Loss: 4.1268
Epoch 29/100, Loss: 0.0759
Epoch 30/100, Loss: 4.9139
Epoch 31/100, Loss: 94.0060
Epoch 32/100, Loss: 0.0303
Epoch 33/100, Loss: 5.0037
Epoch 34/100, Loss: 0.9992
Epoch 35/100, Loss: 0.0010
Epoch 36/100, Loss: 5.8375
Epoch 37/100, Loss: 93.9554
Epoch 

In [17]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
l1 = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        outputs = outputs.squeeze(-1)
        # Accumulate L1
        l1 += l1_loss(outputs, batch_y, reduction='sum').item()
        total += batch_y.size(0)  # Count total samples

# Calculate Mean Absolute Error (average per sample)
l1_error = l1 / total

print(f"Mean Absolute Error Loss (L1) on test data: {l1_error}")

Mean Absolute Error Loss (L1) on test data: 8.52257033756801


## Task 5

### Generate train data with H0 = {hi} and HL

In [2]:
game = GameVisualize()
player_1 = RandomPlayer(True)
player_2 = MiniMaxPlayerWithHFunction(False, depth=DEFAULT_DEPTH, generate_data=True)
x = []
for i in range(1):
    result = game.start_game(player_1, player_2, visual=True, pause=1e-15)

Added row: [0.4784635046163178, 0.2648900630707205, -1.0337863739467426, 1.0934503902199124, 0.9992340531966024]
Added row: [0.6077862946163971, 0.4317132560346112, -1.1647588962875979, 0.7682537070804867, 100]
Added row: [0.20356630567285583, 1.201050977037219, -1.251266945810753, 0.4706801551687476, 1.8837949038552642]
Added row: [0.5968671606229592, 0.44833818935643177, -1.0375154856082003, 0.47200315121943837, 0.9995409077334886]
Added row: [0.05975737375568824, 0.5623718961117876, -1.3143788072088567, 0.8801642778373799, 1.9414960526281035]
Added row: [0.17749796200709977, 0.5848787551422872, -1.0973275609195847, 0.5019891736485984, 5.7590226898050245]
Added row: [0.9550598852563892, 0.5610126559024793, -1.6365889618052814, 0.23677405195333462, 1.806081230371987]
Added row: [0.6434523941102274, -0.04462858223471167, -1.0211021385834291, 0.4425835776607213, 1.118633247659103]
Added row: [0.045006606629355295, 0.435373787112763, -1.0827055122775313, 0.5692852292695089, 0.99655026273

In [18]:
dataset = CSVDataset(DATA_2)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [19]:
model = FlexibleMLP(LAYERS_SIZE_2)
print(model)

# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

FlexibleMLP(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Softmax(dim=1)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Softmax(dim=1)
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): Softmax(dim=1)
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [ ]:
# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "model_2.pth")

/Users/ntd432/Desktop/SE4GD/UnivAQ/AI/code/ai-univaq/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1/1000, Loss: 47.4238
Epoch 2/1000, Loss: 44.5764
Epoch 3/1000, Loss: 65.1183
Epoch 4/1000, Loss: 45.7538
Epoch 5/1000, Loss: 63.5113
Epoch 6/1000, Loss: 64.2622
Epoch 7/1000, Loss: 80.0270
Epoch 8/1000, Loss: 45.7745
Epoch 9/1000, Loss: 28.7621
Epoch 10/1000, Loss: 44.5075
Epoch 11/1000, Loss: 60.3242
Epoch 12/1000, Loss: 58.3597
Epoch 13/1000, Loss: 44.0612
Epoch 14/1000, Loss: 56.8749
Epoch 15/1000, Loss: 68.2388
Epoch 16/1000, Loss: 45.5067
Epoch 17/1000, Loss: 56.1441
Epoch 18/1000, Loss: 56.6328
Epoch 19/1000, Loss: 44.2653
Epoch 20/1000, Loss: 60.7713
Epoch 21/1000, Loss: 37.6186
Epoch 22/1000, Loss: 45.5848
Epoch 23/1000, Loss: 46.6127
Epoch 24/1000, Loss: 38.8794
Epoch 25/1000, Loss: 48.6141
Epoch 26/1000, Loss: 45.7794
Epoch 27/1000, Loss: 42.6556
Epoch 28/1000, Loss: 43.7559
Epoch 29/1000, Loss: 42.6169
Epoch 30/1000, Loss: 45.5148
Epoch 31/1000, Loss: 46.8551
Epoch 32/1000, Loss: 45.7626
Epoch 33/1000, Loss: 42.5465
Epoch 34/1000, Loss: 41.5090
Epoch 35/1000, Loss: 45

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
l1 = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        outputs = outputs.squeeze(-1)
        # Accumulate L1
        l1 += l1_loss(outputs, batch_y, reduction='sum').item()
        total += batch_y.size(0)  # Count total samples

# Calculate Mean Absolute Error (average per sample)
l1_error = l1 / total

print(f"Mean Absolute Error Loss (L1) on test data: {l1_error}")

22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759643555
22.074506759